In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:
df = pd.read_csv('docs/V8.csv', sep = ';')

In [4]:
df.shape

(1804, 15)

In [5]:
df.isnull().sum()

activityId                    0
workerId                      3
date                          0
companyName                   0
establishmentName             0
numberOfRuptures              0
numberProductsOutStorage      0
numberProductsOutShelf        0
rateRuptures                  0
rateOutStorage                0
mixSize                       0
duration                    513
numberOfPictures              0
averageRating                 0
marker                        0
dtype: int64

In [6]:
df.shape

(1804, 15)

In [7]:
#df['duration'] = df.duration.fillna(0)
df = df.dropna()

df2 = df.sample(800)
df3 = df.sample(200)

In [8]:
#Separando dados e cassificador
df_columns = df2.columns

X_train = df2[df_columns[(df_columns != 'marker') & (df_columns != 'averageRating') & (df_columns != 'activityId') & (df_columns != 'workerId') & (df_columns != 'date')]]
y_train = df2['marker']

X_train.head()

,companyName,establishmentName,numberOfRuptures,numberProductsOutStorage,numberProductsOutShelf,rateRuptures,rateOutStorage,mixSize,duration,numberOfPictures
1297,AMABENE (CÂMARA FRIA),Condor - Curitiba - Água Verde,1,4,1,0.17,0.67,6,61.0,6
1249,LOMAN COGUMELOS,Super Muffato - Cascavel - Comercial,4,3,6,0.40,0.30,10,29.0,2
240,TRINDADE MASSAS (CÂMARA FRIA),Mercado Santos - Santa Candida,10,22,10,0.43,0.96,23,24.0,4
881,VINHOS TERRAUSTRAL,Condor - Apucarana,11,16,17,0.48,0.70,23,174.0,21
122,COCA-COLA FEMSA,Condor - Araucária - BR,87,176,98,0.28,0.56,314,53.0,54


In [9]:
#Criar label encoder para váriaveis categoricas
le = preprocessing.LabelEncoder()
X_train['companyName'] = le.fit_transform(X_train['companyName'].astype(str))
X_train['establishmentName'] = le.fit_transform(X_train['establishmentName'].astype(str))
X_train.head()

,companyName,establishmentName,numberOfRuptures,numberProductsOutStorage,numberProductsOutShelf,rateRuptures,rateOutStorage,mixSize,duration,numberOfPictures
1297,0,53,1,4,1,0.17,0.67,6,61.0,6
1249,20,119,4,3,6,0.40,0.30,10,29.0,2
240,38,86,10,22,10,0.43,0.96,23,24.0,4
881,41,19,11,16,17,0.48,0.70,23,174.0,21
122,6,29,87,176,98,0.28,0.56,314,53.0,54


In [10]:
#tranformar classificador em categorias
y_train = to_categorical(y_train)

num_classes = y_train.shape[1]
print(num_classes)

2


In [11]:
#Dataset teste
X_teste = df3[df_columns[(df_columns != 'marker') & (df_columns != 'averageRating') & (df_columns != 'activityId') & (df_columns != 'workerId') & (df_columns != 'date')]]
y_teste = df3['marker']
y_teste = to_categorical(y_teste)

X_teste['companyName'] = le.fit_transform(X_teste['companyName'].astype(str))
X_teste['establishmentName'] = le.fit_transform(X_teste['establishmentName'].astype(str))

X_teste.head()

,companyName,establishmentName,numberOfRuptures,numberProductsOutStorage,numberProductsOutShelf,rateRuptures,rateOutStorage,mixSize,duration,numberOfPictures
3,25,104,9,16,9,0.56,1.00,16,13.0,6
555,14,89,9,10,9,0.90,1.00,10,15.0,4
1038,15,43,0,0,0,0.00,0.00,3,59.0,6
535,11,108,79,80,80,0.92,0.93,86,28.0,4
481,18,71,6,6,6,0.75,0.75,8,38.0,2


In [12]:
#Normalizar dados
X_train_norm = (X_train - X_train.mean()) / X_train.std()
X_teste_norm = (X_teste - X_teste.mean()) / X_teste.std()

In [13]:
#classification model
def classification_model():
    model = Sequential()
    model.add(Dense(10, activation = 'relu', input_shape = (10,)))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(10, activation = 'relu'))

    #model.add(Flatten())
    model.add(Dense(2, activation = 'softmax'))

    model.compile(optimizer='adam', loss= 'categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = classification_model()

In [14]:
model.fit(X_train, y_train, epochs=5, verbose = 2)

model.predict(X_teste) #gives the probablity that a point belongs to each class

Epoch 1/5
 - 0s - loss: 4.0938 - acc: 0.4700
Epoch 2/5
 - 0s - loss: 1.2136 - acc: 0.4488
Epoch 3/5
 - 0s - loss: 0.7777 - acc: 0.5363
Epoch 4/5
 - 0s - loss: 0.7123 - acc: 0.5813
Epoch 5/5
 - 0s - loss: 0.6806 - acc: 0.6013


array([[5.7910669e-01, 4.2089328e-01],
       [5.1425523e-01, 4.8574477e-01],
       [4.3245980e-01, 5.6754017e-01],
       [5.6288940e-01, 4.3711054e-01],
       [5.6243873e-01, 4.3756130e-01],
       [4.7080907e-01, 5.2919090e-01],
       [2.3558070e-01, 7.6441938e-01],
       [5.9708273e-01, 4.0291730e-01],
       [1.5411276e-01, 8.4588724e-01],
       [5.8536243e-01, 4.1463754e-01],
       [5.3129721e-01, 4.6870282e-01],
       [4.4128257e-01, 5.5871749e-01],
       [5.4448873e-01, 4.5551130e-01],
       [7.4602246e-01, 2.5397748e-01],
       [4.4383037e-01, 5.5616957e-01],
       [6.3770747e-01, 3.6229250e-01],
       [5.2832335e-01, 4.7167668e-01],
       [6.9971049e-01, 3.0028948e-01],
       [7.1205533e-01, 2.8794464e-01],
       [4.7607213e-01, 5.2392793e-01],
       [4.5398337e-01, 5.4601663e-01],
       [6.2875539e-01, 3.7124458e-01],
       [5.3064942e-01, 4.6935055e-01],
       [5.3961438e-01, 4.6038565e-01],
       [4.0586790e-01, 5.9413207e-01],
       [6.0989916e-01, 3.

In [15]:
model.evaluate(X_teste, y_teste, verbose=0)

[0.6527097702026368, 0.57]

In [16]:
model.metrics_names


['loss', 'acc']

In [17]:
from keras.utils import plot_model

plot_model(model, to_file='model.png')

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

In [ ]:
#https://keras.io/getting-started/sequential-model-guide/